<a href="https://colab.research.google.com/github/MeghnaAG/Miniproject/blob/main/transcribe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null
!pip install spacy
!pip install PySoundFile
!python -m spacy download en_core_web_md

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.7.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
weasel 0.3.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.12.3
    Uninstalling typer-0.12.3:
      Successfully uninstalled typer-0.12.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyannote-database 5.1.0 requires typer>=0.12.1, but you have typer 0.9.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 31.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load t

In [2]:
!pip install speechbrain==0.5.14

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.0/519.0 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: speechbrain
    Found existing installation: speechbrain 1.0.0
    Uninstalling speechbrain-1.0.0:
      Successfully uninstalled speechbrain-1.0.0


In [3]:
import speechbrain
print(dir(speechbrain))

['Brain', 'Stage', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'alignment', 'core', 'create_experiment_directory', 'dataio', 'decoders', 'f', 'lm', 'lobes', 'nnet', 'os', 'parse_arguments', 'pretrained', 'processing', 'tokenizers', 'utils', 'version']


In [10]:
from google.colab import files
import subprocess
import datetime
import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
from pyannote.audio import Audio
from pyannote.core import Segment
import wave
import contextlib
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import whisper


In [11]:
#from pyannote.audio.features import Pretrained
#from pyannote.audio.pipelines import SpeakerEmbedding
from pyannote.audio.pipelines import SpeakerDiarization
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding

In [16]:

def transcribe_and_cluster_audio(audio_file_path, num_speakers, language='English', model_size='medium'):
    # Ensure that the audio is mono (single-channel)
    subprocess.call(['ffmpeg', '-i', audio_file_path, '-ac', '1', 'audio_mono.wav', '-y'])
    path = 'audio_mono.wav'

    # Determine the appropriate model name based on language and model size
    model_name = model_size
    if language == 'English' and model_size != 'large':
        model_name += '.en'
    if path[-3:] != 'wav':
      subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
      path = 'audio.wav'

    #Convert to mono
    """subprocess.call(['ffmpeg', '-i', path, '-ac', '1', 'audio_mono.wav', '-y'])
    path = 'audio_mono.wav'"""

    # Load the whisper model
    model = whisper.load_model(model_size)

    # Transcribe the audio using the whisper model
    result = model.transcribe(path)
    segments = result["segments"]

    # Open the audio file to get duration
    with contextlib.closing(wave.open(path, 'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)

    audio = Audio()

    # Load the pretrained speaker embedding model
    embedding_model = PretrainedSpeakerEmbedding("speechbrain/spkrec-ecapa-voxceleb", device=torch.device("cuda"))

    def segment_embedding(segment):
        start = segment["start"]
        end = min(duration, segment["end"])
        clip = Segment(start, end)
        waveform, sample_rate = audio.crop(path, clip)
        if waveform.shape[0] > 1:
           waveform = waveform.mean(axis=0, keepdims=True)
        return embedding_model(waveform[None])

    # Extract embeddings for each segment
    embeddings = np.zeros(shape=(len(segments), 192))
    for i, segment in enumerate(segments):
        embeddings[i] = segment_embedding(segment)

    embeddings = np.nan_to_num(embeddings)

    # Cluster the embeddings to identify speakers
    #clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
    #labels = clustering.labels_
    #for i in range(len(segments)):
        #segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

    # Perform clustering
    clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
    labels = clustering.labels_

    # Get unique speaker labels
    unique_labels = np.unique(labels)

    # Initialize a dictionary to map original labels to speaker IDs
    label_mapping = {label: i + 1 for i, label in enumerate(unique_labels)}

    # Assign speaker labels to segments
    for i in range(len(segments)):
        original_label = labels[i]  # Get original cluster label
        speaker_id = label_mapping[original_label]  # Map original label to speaker ID
        segments[i]["speaker"] = 'SPEAKER ' + str(speaker_id)

    clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
    labels = clustering.labels_
    for i in range(len(segments)):
        segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

    def time(secs):
      return datetime.timedelta(seconds=round(secs))

    f = open("transcript.txt", "w")

    for (i, segment) in enumerate(segments):
      if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
          f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
          f.write(segment["text"][1:] + ' ')
    f.close()

    f = open("transcript.txt", "w")

    prev_speaker = None  # Initialize previous speaker variable
    speaker_mapping = {}  # Dictionary to map original cluster labels to speaker IDs
    first_speaker = True  # Flag to indicate whether it's the first speaker

    for (i, segment) in enumerate(segments):
        current_speaker = segment["speaker"]  # Get current speaker label
        text = segment["text"][1:]  # Get text of the segment

    # Assign speaker IDs sequentially
        if current_speaker not in speaker_mapping:
            speaker_mapping[current_speaker] = len(speaker_mapping) + 1

        speaker_id = speaker_mapping[current_speaker]  # Get speaker ID

    # Format the output
        if not first_speaker and prev_speaker != current_speaker:
            f.write(f"\nSPEAKER {speaker_id}: {text} ")  # Add newline for non-first speakers
        else:
            f.write(f"SPEAKER {speaker_id}: {text} ")  # No newline for the first speaker
            first_speaker = False  # Update first speaker flag

        prev_speaker = current_speaker  # Update previous speaker

    f.close()




    # Write the transcribed text along with speaker labels to a file
    #def time(secs):
     #   return datetime.timedelta(seconds=round(secs))

    #with open("transcript.txt", "w") as f:
     #   for (i, segment) in enumerate(segments):
      #      if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
       #         f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
        #    f.write(segment["text"][1:] + ' ')

    print("Transcription and clustering completed. Results saved to 'transcript.txt'.")


In [13]:
import spacy
import re

def extract_speaker_names(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    names = list(set([ent.text.strip() for ent in doc.ents if ent.label_ == 'PERSON']))
    return names

def map_speakers(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    # Initialize variables
    current_speaker = None
    current_lines = []
    speaker_names = {}

# Define a regular expression pattern for speaker labels
    speaker_label_pattern = re.compile(r"^(SPEAKER \d+):")
# Define a regular expression pattern for speaker introductions
    speaker_pattern = re.compile(r"^(Good morning everyone, I'm|Hi,? everyone,? I am|Hi,? everyone,? I'm|Hi,? I am|Hi,? I'm|I'm) ([^.]+)\.?$")

# Iterate through the sentences
    for sent in doc.sents:
    # Check if the sentence contains a speaker label
        label_match = speaker_label_pattern.match(sent.text)
        if label_match:
            current_speaker = label_match.group(1)
            sent_text = sent.text.replace(label_match.group(0), '').strip()
        # Check if the sentence matches the speaker pattern
            intro_match = speaker_pattern.match(sent_text)
            if intro_match:
                current_speaker_name = intro_match.group(2)
                current_lines.append(f"{current_speaker_name}: {sent_text.replace(intro_match.group(0), '').strip()}")
                speaker_names[current_speaker] = current_speaker_name
            else:

                if current_speaker in speaker_names:
                    current_lines.append(f"{speaker_names[current_speaker]}: {sent_text}")
                else:
                    current_lines.append(sent.text.strip())

        else:
            if current_speaker in speaker_names:
                current_lines[-1] += f" {sent.text.strip()}"
            else:
                current_lines.append(sent.text.strip())


    return current_lines


In [17]:
from google.colab import files
# Upload your audio file
uploaded = files.upload()

# Get the file name
audio_file_name = list(uploaded.keys())[0]
#path = next(iter(uploaded))
# Call the function to transcribe audio
#transcription_result = transcribe_and_cluster_audio(audio_file_name)
num_speakers = int(input("Enter the number of speakers: "))

# Call the function to transcribe audio
transcribe_and_cluster_audio(audio_file_name, num_speakers=num_speakers)

# Save the transcription to a text file with the same name as the audio file
#output_text_file = f"{audio_file_name.split('.')[0]}.txt"
#with open(output_text_file, "r") as f:
    # file_content = f.read()
     #print(f"Content of {output_text_file}:\n{file_content}")

#print(f"Transcription saved to {output_text_file}")

# Map speakers using the saved transcription file
#speaker_lines = map_speakers(output_text_file)
#for line in speaker_lines:
 #   print(line)
input_file_path = 'transcript.txt'  # Replace with your actual file path
with open(input_file_path, "r") as file:
    file_content = file.read()

# Extract speaker names
speaker_names = extract_speaker_names(file_content)
print(f"Speaker Names: {speaker_names}")

# Map speakers using the content from the file
speaker_lines = map_speakers(file_content)
for line in speaker_lines:
    print(line)

Saving Trial_1.wav to Trial_1 (4).wav
Enter the number of speakers: 2


100%|██████████████████████████████████████| 1.42G/1.42G [00:09<00:00, 154MiB/s]


hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

Transcription and clustering completed. Results saved to 'transcript.txt'.
Speaker Names: ['James', 'Sarah']
Sarah:  Let's jump right in. James, quick update on marketing.
James:  Sure, we have seen a 20% increase in website traffic and social media engagement is up by 15%.
